# 수원서베이 2024 자동 EDA + 핵심지표 대시보드

- 입력: `/mnt/data/suwon_2024_labeled.(xlsx|csv)`
- 출력: `/mnt/data/eda/dashboard.xlsx` (summary + 변수별 시트)
- 가중치: `ws` 우선, 없으면 `wg`, 둘 다 없으면 비가중치
- 공통 문항: `/mnt/data/common_vars.txt`(한 줄당 1개) 있으면 사용, 없으면 기본 리스트 사용


## 0) 로드/설정

In [1]:

from pathlib import Path
import pandas as pd
import numpy as np
import math

BASE = Path.cwd().parent
DATA_XLSX = BASE / "output" / "1. 수원서베이" / "suwon_2024_labeled.xlsx"
DATA_CSV = BASE / "output" / "1. 수원서베이" / "suwon_2024_labeled.csv"
OUTDIR = BASE / "output" / "1. 수원서베이" / "eda"
OUTDIR.mkdir(parents=True, exist_ok=True)
DASHBOARD_PATH = OUTDIR / "dashboard.xlsx"

# 1) 데이터 로드
if DATA_XLSX.exists():
    df = pd.read_excel(DATA_XLSX, sheet_name="data_labeled")
elif DATA_CSV.exists():
    df = pd.read_csv(DATA_CSV)
else:
    raise FileNotFoundError("라벨링 데이터 파일이 없습니다. suwon_2024_labeled.xlsx 또는 CSV를 확인하세요.")

print("Loaded:", df.shape)

# 2) 공통 문항 로드
common_txt = BASE / "common_vars.txt"
if common_txt.exists():
    with open(common_txt, "r", encoding="utf-8") as f:
        COMMON_VARS = [line.strip() for line in f if line.strip()]
else:
    # 기본 예시(라벨링 후 컬럼명 기준) — 필요 시 교체
    COMMON_VARS = [
        "정책 관심도",
        "수원시정 만족도",
        "한 주간 삶의 질_평균(100점)",
        "영역별 행복 정도_평균(100점)",
        "환경 만족도_평균(100점)"
    ]

# 3) 가중치 선택
WEIGHT_COL = None
if "ws" in df.columns:
    WEIGHT_COL = "ws"
elif "wg" in df.columns:
    WEIGHT_COL = "wg"

print("Weight column:", WEIGHT_COL)
print("Common vars (candidate):", COMMON_VARS[:10])


Loaded: (3057, 481)
Weight column: ws
Common vars (candidate): ['정책 관심도', '수원시정 만족도', '한 주간 삶의 질_평균(100점)', '영역별 행복 정도_평균(100점)', '환경 만족도_평균(100점)']


## 1) 보조 함수

In [2]:

LIKERT_KEYWORDS = [
    "전혀", "그렇지 않다", "보통", "그렇다", "매우", "만족", "불만족", "동의", "비동의",
    "낮다", "높다", "나쁘다", "좋다", "의견", "정도", "점수", "만큼"
]

def looks_like_likert(series, sample_k=30):
    vals = series.dropna().astype(str).unique()[:sample_k]
    hit = 0
    for v in vals:
        if any(k in v for k in LIKERT_KEYWORDS):
            hit += 1
    nunique = series.nunique(dropna=True)
    return (4 <= nunique <= 7) and (hit >= max(1, int(np.ceil(len(vals) * 0.2))))

def infer_var_type(s, cat_threshold=0.05, max_cat_unique=30):
    s_num = pd.to_numeric(s, errors="coerce")
    numeric_ratio = s_num.notna().mean()
    nunique = s.nunique(dropna=True)
    if numeric_ratio > 0.98:
        return "numeric"
    if looks_like_likert(s):
        return "ordinal_likert"
    if nunique <= max_cat_unique or nunique / max(1, len(s)) <= cat_threshold:
        return "categorical"
    return "categorical"

def summarize_numeric(s):
    s_num = pd.to_numeric(s, errors="coerce")
    return {
        "count": int(s_num.count()),
        "mean": float(s_num.mean()) if s_num.count() else np.nan,
        "std": float(s_num.std()) if s_num.count() else np.nan,
        "min": float(s_num.min()) if s_num.count() else np.nan,
        "q25": float(s_num.quantile(0.25)) if s_num.count() else np.nan,
        "median": float(s_num.median()) if s_num.count() else np.nan,
        "q75": float(s_num.quantile(0.75)) if s_num.count() else np.nan,
        "max": float(s_num.max()) if s_num.count() else np.nan,
        "nunique": int(s_num.nunique(dropna=True)),
        "na_rate": float(s.isna().mean())
    }

def weighted_mean(x, w):
    x = pd.to_numeric(x, errors="coerce")
    m = ~x.isna() & ~w.isna()
    if m.sum() == 0:
        return np.nan
    return float((x[m] * w[m]).sum() / w[m].sum())

def weighted_std(x, w):
    x = pd.to_numeric(x, errors="coerce")
    m = ~x.isna() & ~w.isna()
    if m.sum() < 2:
        return np.nan
    mu = (x[m] * w[m]).sum() / w[m].sum()
    var = ((w[m] * (x[m] - mu) ** 2).sum()) / (w[m].sum())
    return float(np.sqrt(var))

def numeric_stats(s, w=None):
    base = summarize_numeric(s)
    if w is not None:
        wm = weighted_mean(s, w)
        ws = weighted_std(s, w)
        base.update({"w_mean": wm, "w_std": ws})
    return base

def categorical_freq(s, w=None):
    s2 = s.fillna("(결측)").astype(str)
    if w is None:
        vc = s2.value_counts(dropna=False).rename("count").to_frame()
        vc["ratio"] = vc["count"] / len(s2)
    else:
        w2 = w.reindex(s.index)
        tmp = pd.DataFrame({"v": s2, "w": w2})
        grp = tmp.groupby("v", dropna=False, as_index=False)["w"].sum()
        total = grp["w"].sum()
        grp = grp.rename(columns={"w": "weight"}).sort_values("weight", ascending=False)
        grp["ratio"] = grp["weight"] / total if total > 0 else np.nan
        grp = grp.rename(columns={"v": "value"})
        grp["count"] = np.nan  # 비가중 count는 의미 없음
        return grp[["value", "ratio", "count", "weight"]]
    vc = vc.reset_index()
    vc.columns = ["value", "count", "ratio"]
    return vc[["value", "ratio", "count"]]


## 2) 대시보드 생성 및 저장

In [3]:

# 대시보드 생성
summary_rows = []
missing_vars = []

# 가중치 시리즈 준비
w = None
if WEIGHT_COL is not None and WEIGHT_COL in df.columns:
    w = pd.to_numeric(df[WEIGHT_COL], errors="coerce")

with pd.ExcelWriter(DASHBOARD_PATH, engine="xlsxwriter") as writer:
    for var in COMMON_VARS:
        if var not in df.columns:
            missing_vars.append(var)
            continue
        s = df[var]
        vtype = infer_var_type(s)

        safe_sheet = var[:31]  # Excel sheet name limit
        if vtype == "numeric":
            stats = numeric_stats(s, w)
            stats["variable"] = var
            stats["type"] = "numeric"
            if w is not None:
                stats["weight"] = WEIGHT_COL
            pd.DataFrame([stats]).to_excel(writer, sheet_name=safe_sheet, index=False)
            summary_rows.append(stats)
        else:
            freq = categorical_freq(s, w)
            freq.to_excel(writer, sheet_name=safe_sheet, index=False)
            # summary top-1
            if not freq.empty:
                top = freq.iloc[0].to_dict()
                summary_rows.append({
                    "variable": var,
                    "type": "categorical" if vtype != "ordinal_likert" else "ordinal_likert",
                    "top1_value": top.get("value"),
                    "top1_ratio": float(top.get("ratio")) if pd.notna(top.get("ratio")) else np.nan,
                    "nunique": int(s.nunique(dropna=True)),
                    "weight": WEIGHT_COL if w is not None else None
                })
            else:
                summary_rows.append({
                    "variable": var,
                    "type": "categorical" if vtype != "ordinal_likert" else "ordinal_likert",
                    "top1_value": None,
                    "top1_ratio": np.nan,
                    "nunique": int(s.nunique(dropna=True)),
                    "weight": WEIGHT_COL if w is not None else None
                })

    # summary 시트 저장
    pd.DataFrame(summary_rows).to_excel(writer, sheet_name="summary", index=False)

print("Dashboard saved to:", DASHBOARD_PATH)
if missing_vars:
    print("[WARN] 존재하지 않는 변수명:", missing_vars)


Dashboard saved to: d:\workspace\dacon_sri\output\1. 수원서베이\eda\dashboard.xlsx
[WARN] 존재하지 않는 변수명: ['정책 관심도', '수원시정 만족도', '한 주간 삶의 질_평균(100점)', '영역별 행복 정도_평균(100점)', '환경 만족도_평균(100점)']
